In [1]:
import pandas as pd
import numpy as np
import sys
import pickle
import warnings
warnings.filterwarnings("ignore")

sys.path.insert(0, '..')
from DataModule.Data_Preparation import CoronnaCERTAINDataset
import EvaluationModule

import xgboost as xgb
import sklearn
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import ShuffleSplit,cross_val_score
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.pipeline import Pipeline

import submitit

(CVXPY) Jul 07 10:50:06 PM: Encountered unexpected exception importing solver CVXOPT:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 07 10:50:06 PM: Encountered unexpected exception importing solver GLPK:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so, 0x0002): tried: '/Users/gaskell/miniforge_x86_64/envs/ADPred_new/lib/python3.10/site-packages/cvxopt/base.cpython-310-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e'))")
(CVXPY) Jul 07 10:50:06 PM: Encountered unexpected exception importing solver GLPK_MI:
ImportError("dlopen(/Users/gaskell/miniforge_x86_64/e

In [2]:
dataset = CoronnaCERTAINDataset(
    library_root='/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
    challenge="regression_delta", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='KVB', #option: KVB, SC
    imputation='SimpleFill', #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='KVB', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    remove_low_DAS = True,
    save_csv=False, 
    random_state=2022)

In [3]:
# read train, test from dataloader
train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()

In [4]:
# get x,y train and test set
X_train = train.iloc[:,:-1] # train.iloc[:,-2]
y_train = train.iloc[:,-1] 
X_test = test.iloc[:,:-1] # test.iloc[:,-2]
y_test = test.iloc[:,-1]

In [5]:
def main():
    remove_low_DAS_list = [True, False]
#     remove_low_DAS_list = [True]
    challenge_list = ["regression", "regression_delta", "classification", "binary_classification"]
#     challenge_list = ["classification", "binary_classification"]
    imputation_list = ["SimpleFill", "KNN", "SoftImpute", "IterativeImputer"]
#     imputation_list = ["KNN"]
    process_approach_list = ['KVB','SC']
#     process_approach_list = ['SC']
    patient_group_list = ["all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"]
#     patient_group_list = ["bionaive TNF"]
    for remove_low_DAS in remove_low_DAS_list:
        for challenge in challenge_list:
            for imputation in imputation_list:
                for process_approach in process_approach_list:
                    for patient_group in patient_group_list:
                        print("remove_low_DAS",remove_low_DAS)
                        print("clallenge:",challenge)
                        print("imputation:", imputation)
                        print("process_approach", process_approach)
                        print("patient_group", patient_group)
                        dataset = CoronnaCERTAINDataset(
                            library_root = '/Users/gaskell/Dropbox/Mac/Desktop/Autoimmune_Disease/Code/ML_RA_EHR/Dataset/',
                            challenge = challenge, #option: regression, regression_delta, classification, binary_classification
                            dataset = 'CORRONA CERTAIN', 
                            process_approach = process_approach, #option: KVB, SC
                            imputation = imputation, #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
                            patient_group =patient_group, #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
                            drug_group = 'all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
                            time_points = (0,3), 
                            train_test_rate = 0.8,
                            remove_low_DAS = remove_low_DAS,
                            save_csv = False, 
                            random_state = 2022)
                        # read train, test from dataloader
                        train_set, train_loc = dataset.get_train()
                        test_set, test_loc = dataset.get_test()
                        
                        aml = EvaluationModule.AutoBuild(seed=dataset.random_state, project_name="SC_Jul8_2", challenge=dataset.challenge)
                        # define models
                        if "regression" in dataset.challenge:
                            model = RandomForestRegressor()
                        elif "classification" in dataset.challenge:
                            model = RandomForestClassifier()
                        aml.validate("rf", model, train_set, test_set)
                        aml.validation_output(dataset)
                        aml.test_output(dataset)

In [6]:
main()

remove_low_DAS True
clallenge: regression
imputation: SimpleFill
process_approach KVB
patient_group all
remove_low_DAS True
clallenge: regression
imputation: SimpleFill
process_approach KVB
patient_group bioexp nTNF
remove_low_DAS True
clallenge: regression
imputation: SimpleFill
process_approach KVB
patient_group bionaive TNF
remove_low_DAS True
clallenge: regression
imputation: SimpleFill
process_approach KVB
patient_group bionaive orencia
remove_low_DAS True
clallenge: regression
imputation: SimpleFill
process_approach KVB
patient_group KVB
remove_low_DAS True
clallenge: regression
imputation: SimpleFill
process_approach SC
patient_group all
feature engineering, drop columns due to 70% missing value: Index(['smkyrs', 'numcigs', 'rfstatus_impute', 'ccpstatus_impute',
       'statin_use'],
      dtype='object')
Missing values in train before imputation: 1079
Missing values in train after imputation: 0
Missing values in test before imputation: 249
Missing values in test after imputatio